Encountered a problem with my wrapper function. 
My function works for two models but this one uses lesser dimensions.

In [23]:
import sys
sys.path.append('./see-segment')

In [24]:
from transformers import AutoProcessor, CLIPSegForImageSegmentation
from PIL import Image
import requests
from wrapper import logit_image

processor = AutoProcessor.from_pretrained("CIDAS/clipseg-rd64-refined")
model = CLIPSegForImageSegmentation.from_pretrained("CIDAS/clipseg-rd64-refined")
image_path = 'Images/trump.png'
image = Image.open(image_path).convert("RGB")
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)
texts = ["a cat", "a remote", "a blanket"]
inputs = processor(text=texts, images=[image] * len(texts), padding=True, return_tensors="pt")

outputs = model(**inputs)

logits = outputs.logits
image = logit_image.logits_to_image(logits)
image.show()


IndexError: too many indices for tensor of dimension 3